# Advent of Code 2023

## Day 5 -- If You Give A Seed A Fertilizer (Part 2)

## Author: Chris Kimber

The instructions for this problem can be found at https://adventofcode.com/2023/day/5.

Note: The challenge for part 2 is that instead of having 20 input values for the first map ("seeds"), there are 10 ranges of input values. This moves the number of input values into the billions and makes a brute-force solution rather infeasible. Instead of solving this totally solo, I did look at some reddit comments on the problem though I did not borrow any code. The 'optimal' way to solve this problem is by splitting the input ranges to create blocks of values that move through the maps together. Instead, I used a hack of sorts. Because the goal is to find the minimum final value ("location") that can be produced by one of the input values, you can run through the maps backwards starting at location 0 and incrementing the location by 1 until a value matching a seed within range is found. While this is still brute force, it should be computationally quicker than running through all the available seed values. I will hopefully try to solve the problem using range splitting later.

The code to load in the data and parse the seed values is repeated from part 1.

In [79]:
with open("input", "r") as f:
    input_file = f.read().rstrip().split("\n\n")

In [80]:
input_list = [x.split("\n") for x in input_file]

In [81]:
seeds = input_list.pop(0)

In [82]:
seeds = [x.split(" ")[1:] for x in seeds]

In [83]:
seeds = [int(number) for x in seeds for number in x]

There are cleaner ways to do this, but here the seed values are parsed into ranges by identifying which indices are range starts and then generating ranges from these indices using the function below.

In [85]:
start_indices = range(0, len(seeds), 2)

In [8]:
def calc_seed_ranges(seeds, start_indices):
    seed_ranges = []
    for i in start_indices:
        seed_range = range(seeds[i], seeds[i]+seeds[i+1])
        seed_ranges.append(seed_range)
    return seed_ranges

In [86]:
seed_ranges = calc_seed_ranges(seeds, start_indices)

The parsing of the remaining data into a series of input-output maps is also repeated from part 1.

In [87]:
input_list = [x[1:] for x in input_list]

In [88]:
input_list = [[string.split(" ") for string in line] for line in input_list]

In [36]:
def make_ranges(map):
    source_range = range(int(map[1]), int(map[1])+int(map[2]))
    destination_range = range(int(map[0]), int(map[0])+int(map[2]))
    return [source_range, destination_range]

In [89]:
input_list = [[make_ranges(map) for map in line] for line in input_list]

The following function is derived from its equivalent in part 1 as well, but the logic is reversed so the second range in each offset map is the input and the first the output (as the maps are being traversed in reverse).

In [17]:
def map_processor(num, map_doc):
    for i in map_doc:
        if num in i[1]:
            diff = num-i[1][0]
            destination = i[0][0] + diff
            output_num = destination
            break
    else:
        output_num = num
    return output_num      

The following function takes the value at the end of the reversed map traversal and checks to see if it is in any of the ranges of seed values provided.

In [73]:
def seed_checker(num, seed_ranges):
    for seed in seed_ranges:
        if num in seed:
            return True
            break

The following function runs through the list of maps in reverse from an initial value and then checks to see if the final output was in the range of seed values provided.

In [20]:
def location_to_seed(num, input_list):
    for map_doc in reversed(input_list):
        output_num = map_processor(num, map_doc)
        num = output_num
    return seed_checker(output_num, seed_ranges)   

All the functions above are used in a while loop to increment through starting values (locations) from 0 until a location matching a seed in one of the provided ranges is found. When one is found, the function nested in the loop will return True. This is a little clunky because the start value is being incremented in the while loop, so it will have advanced by 1 before the condition is found to be True at the start of the next loop iteration. As such, the value is decreased by 1 to find the lowest location value that matched a provided seed; this is the answer to the problem!

In [93]:
start_num = 0
in_range = False
while in_range != True:
    in_range = location_to_seed(start_num, input_list)
    start_num = start_num + 1
lowest_location = start_num - 1

In [92]:
print(lowest_location)

34039469